### Load Data

In [1]:
import os
import numpy as np
# from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier
import catboost as cb
from sklearn.metrics import accuracy_score, classification_report

def upload_file(filename):
    # Check if the file exists in the current directory
    if os.path.isfile(filename):
        print(f"{filename} exists in the current directory.")
    else:
        print(f"{filename} does not exist. Please upload the file.")
        uploaded = files.upload()

        # Check if the uploaded file is the one we're looking for
        if filename in uploaded:
            print(f"{filename} has been uploaded successfully.")
        else:
            print(f"{filename} was not uploaded. Please try again.")

    return filename

# Example usage:
csv_filename = '/kaggle/input/amexxxx/train_data_with_samplefeatures.csv'
csv_team1_filename = '/kaggle/input/amexxxx/team1_avg_data.csv'
csv_ground_filename = '/kaggle/input/amexxxx/ground_avg_data.csv'
csv_test_filename = '/kaggle/input/r2data/r2_data_with_samplefeatures.csv'

csv_filename = upload_file(csv_filename)
csv_team1_filename = upload_file(csv_team1_filename)
csv_ground_filename = upload_file(csv_ground_filename)
csv_test_filename = upload_file(csv_test_filename)


# Now you can read the CSV file into a pandas DataFrame
data = pd.read_csv(csv_filename)
team1_data=pd.read_csv(csv_team1_filename)
ground_data=pd.read_csv(csv_ground_filename)
test_data = pd.read_csv(csv_test_filename)

/kaggle/input/amexxxx/train_data_with_samplefeatures.csv exists in the current directory.
/kaggle/input/amexxxx/team1_avg_data.csv exists in the current directory.
/kaggle/input/amexxxx/ground_avg_data.csv exists in the current directory.
/kaggle/input/r2data/r2_data_with_samplefeatures.csv exists in the current directory.


In [2]:
df_batsman = pd.read_csv('/kaggle/input/amexxxx/batsman_level_scorecard.csv')
df_bowler = pd.read_csv('/kaggle/input/amexxxx/bowler_level_scorecard.csv')
df_match = pd.read_csv('/kaggle/input/matchhh/match_level_scorecard.csv')

In [3]:
data

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,...,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,Hl Ph,12634,Hl Ph,...,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.00,157.178571
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409....,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:161392...,Ed,20,Wt Is,...,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.00,103.500000
2,9433269,We,10576,3298427.0:2288789.0:7773338.0:3519011.0:368195...,Ne,8987,4003390.0:1749075.0:1626526.0:4172447.0:551672...,We,10576,Ne,...,2023-06-02,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.00,154.333333
3,9587073,Ga An Ws,36084,8127230.0:4690328.0:4069666.0:7960847.0:469018...,Bs Rs,36070,3462080.0:2436405.0:1798705.0:7550857.0:574247...,Ga An Ws,36084,Ga An Ws,...,2023-09-10,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.00,144.250000
4,9516457,Pb Ks,30407,8127181.0:197658.0:4239038.0:2398346.0:5053082...,Gt Ts,48341,1958683.0:7491224.0:8059029.0:4377610.0:225245...,Gt Ts,48341,Gt Ts,...,2023-04-13,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.00,189.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9128601,Pb Ks,30407,2789079.0:197658.0:2398346.0:2827327.0:2082044...,Ci Sr Ks,30414,7422673.0:249087.0:3519011.0:5001170.0:232000....,Pb Ks,30407,Ci Sr Ks,...,2022-04-25,night match,In Pr Le,2022,5004,0.823529,1.000000,147.333333,66.67,166.400000
944,9433241,Mx,8700,4421689.0:7752989.0:1941743.0:4489722.0:767287...,St,9701,6139370.0:7694581.0:3294444.0:3239102.0:632036...,St,9701,St,...,2023-06-02,day/night match,Vy Bt,2023,1042,1.571429,0.012346,167.400000,0.00,170.466667
945,9097227,Bd,22497,4239773.0:1941743.0:3007969.0:4172972.0:155625...,Wn Pe,23869,323049.0:4876122.0:4164978.0:1837205.0:3373138...,Wn Pe,23869,Bd,...,2022-02-07,day match,CA T0 Ce,2021/22,1224,3.000000,1.000000,NaN,0.00,NaN
946,9516695,Rn Rs,30428,8058959.0:2162782.0:2981614.0:4690188.0:212569...,Ss Hd,36014,5958840.0:7491294.0:3127354.0:3057312.0:420349...,Ss Hd,36014,Rn Rs,...,2023-05-07,night match,In Pr Le,2023,4661,0.789474,1.487805,182.800000,66.67,133.375000


In [4]:
# def giveLastNgamesPlayer(player_id, date, n, bat_or_bowl):
#     if bat_or_bowl == 'bat':
#         # Read the CSV file
#         df = df_batsman
        
#         # Convert date strings to datetime objects
#         df['match_dt'] = pd.to_datetime(df['match_dt'])
#         input_date = pd.to_datetime(date)
#         # Filter the DataFrame for the specific player and dates before the input date
#         player_df = df[(df['batsman_id'] == player_id) & (df['match_dt'] < input_date)]
        
#         # Sort the DataFrame by date in descending order
#         player_df = player_df.sort_values('match_dt', ascending=False)
        
#         # Get the last n unique matches
#         last_n_matches = player_df.head(n)
        
#         # Return the full rows for these matches, maintaining the original order
#         result = player_df[player_df['match id'].isin(last_n_matches['match id'])].sort_values('match_dt', ascending=False)
        
#         return result
    
#     elif bat_or_bowl == 'bowl':
#         # Read the CSV file
#         df = df_bowler
        
#         # Convert date strings to datetime objects
#         df['match_dt'] = pd.to_datetime(df['match_dt'])
#         input_date = pd.to_datetime(date)
#         # Filter the DataFrame for the specific player and dates before the input date
#         player_df = df[(df['bowler_id'] == player_id) & (df['match_dt'] < input_date)]
        
#         # Sort the DataFrame by date in descending order
#         player_df = player_df.sort_values('match_dt', ascending=False)
        
#         # Get the last n unique matches
#         last_n_matches = player_df.head(n)
        
#         # Return the full rows for these matches, maintaining the original order
#         result = player_df[player_df['match id'].isin(last_n_matches['match id'])].sort_values('match_dt', ascending=False)
        
#         return result
        
    
#     else:
#         raise ValueError("bat_or_bowl must be either 'bat' or 'bowl'")

In [5]:
# def powerHitters(df_match, match_id, team_seq, date, n):
#     player_list = df_match[df_match['match id']==match_id][f'team{team_seq}_roster_ids'].values[0]
#     player_list = [x for x in str(player_list).split(':')]
#     res_list = []
#     for player in player_list:
#         df_rel = giveLastNgamesPlayer(float(player), date, n, 'bat')
#         df_hit = df_rel[(df_rel['balls_faced']>=10) & (df_rel['strike_rate']>=150)]
#         res_list.append(len(df_hit))

#     isHitter = (np.array(res_list) >= 2).sum()
#     return isHitter

In [6]:
# from tqdm import tqdm
# tqdm.pandas()

In [7]:
# window = [5]
# for i in window:
#     data['team1_powerHitterCount_in_last_{}'.format(i)] = data.progress_apply(lambda x: powerHitters(df_match = data, match_id=x['match id'], team_seq=1, date=x['match_dt'], n=i), axis=1)
#     data['team2_powerHitterCount_in_last_{}'.format(i)] = data.progress_apply(lambda x: powerHitters(df_match = data,match_id=x['match id'], team_seq=2, date=x['match_dt'], n=i), axis=1)

#     test_data['team1_powerHitterCount_in_last_{}'.format(i)] = test_data.progress_apply(lambda x: powerHitters(df_match = test_data,match_id=x['match id'], team_seq=1, date=x['match_dt'], n=i), axis=1)
#     test_data['team2_powerHitterCount_in_last_{}'.format(i)] = test_data.progress_apply(lambda x: powerHitters(df_match = test_data,match_id=x['match id'], team_seq=2, date=x['match_dt'], n=i), axis=1)

In [8]:
def get_team1_name(df, team_id):
    team_names = df.loc[df['team1_id'] == team_id, 'team1']
    if(team_names.empty):
        return "null"
    else:
        team_name = team_names.iloc[0]
        return team_name

def get_team2_name(df, team_id):
    team_names = df.loc[df['team2_id'] == team_id, 'team2']
    if(team_names.empty):
        return "null"
    else:
        team_name = team_names.iloc[0]
        return team_name

def calculate_win_rate(team_id, data):
    total_matches = data[(data['team1_id'] == team_id) | (data['team2_id'] == team_id)].shape[0]
    wins = data[((data['winner'] == get_team1_name(df_match, team_id)) & (data['team1_id'] == team_id)) |
                ((data['winner'] == get_team2_name(df_match, team_id)) & (data['team2_id'] == team_id))].shape[0]
    return wins / total_matches if total_matches > 0 else 0

data['team1_win_rate'] = data['team1_id'].apply(lambda x: calculate_win_rate(x, df_match))
data['team2_win_rate'] = data['team2_id'].apply(lambda x: calculate_win_rate(x, df_match))

In [9]:
def venue_performance(team_id, venue_id, data):
    venue_matches = data[(data['ground_id'] == venue_id) & 
                         ((data['team1_id'] == team_id) | (data['team2_id'] == team_id))]
    wins = venue_matches[((venue_matches['winner'] == get_team1_name(df_match, team_id)) & (venue_matches['team1_id'] == team_id)) |
                         ((venue_matches['winner'] == get_team2_name(df_match, team_id)) & (venue_matches['team2_id'] == team_id))].shape[0]
    return wins / venue_matches.shape[0] if venue_matches.shape[0] > 0 else 0

data['team1_venue_performance'] = data.apply(lambda x: venue_performance(x['team1_id'], x['ground_id'], df_match), axis=1)
data['team2_venue_performance'] = data.apply(lambda x: venue_performance(x['team2_id'], x['ground_id'], df_match), axis=1)

In [10]:
def recent_performance(team_id, data, n=5):
    # Filter matches where the team participated
    team_matches = data[(data['team1_id'] == team_id) | (data['team2_id'] == team_id)]
    
    # Sort by match date to get the most recent matches
    team_matches = team_matches.sort_values(by='match_dt', ascending=False).head(n)
    
    # Count wins for the team
    wins = team_matches[((team_matches['winner'] == get_team1_name(df_match, team_id)) & (team_matches['team1_id'] == team_id)) |
                        ((team_matches['winner'] == get_team2_name(df_match, team_id)) & (team_matches['team2_id'] == team_id))].shape[0]
    
    # Return the win rate over the last N matches
    return wins / n if n > 0 else 0


# Apply the function to calculate recent performance for team1 and team2
data['team1_recent_performance'] = data['team1_id'].apply(lambda x: recent_performance(x, df_match))
data['team2_recent_performance'] = data['team2_id'].apply(lambda x: recent_performance(x, df_match))

In [11]:
# # Create features for run rate, wickets lost, and balls faced
# df['inning1_run_rate'] = df['inning1_runs'] / (df['inning1_balls'] / 6)
# df['inning2_run_rate'] = df['inning2_runs'] / (df['inning2_balls'] / 6)

# # Create additional features
# df['run_rate_diff'] = df['inning1_run_rate'] - df['inning2_run_rate']
# df['total_runs_scored'] = df['inning1_runs'] + df['inning2_runs']
# df['wickets_diff'] = df['inning1_wickets'] - df['inning2_wickets']

In [12]:
df_match

,match id,team1,team2,winner,by,win amount,toss winner,toss decision,venue,city,...,inning2_runs,inning2_wickets,inning2_balls,team1_id,team1_roster_ids,team2_id,team2_roster_ids,series_type,winner_id,player_of_the_match_id
0,8638034,Nn Ds,Wn,Wn,wickets,9.0,Wn,field,By Ol,Mount Maunganui,...,152.0,1.0,97.0,17982,7907451.0:4381761.0:31464.0:258649.0:4949790.0...,18570,2653993.0:6718326.0:6718382.0:2486896.0:228878...,other_domestic,18570,NaN
1,8588005,Me Rs,Sy Tr,Sy Tr,runs,7.0,Sy Tr,field,Ca Ol,Carrara,...,117.0,2.0,74.0,33942,37351.0:46794.0:5406540.0:2231928.0:181404.0:1...,33963,1506098.0:1749075.0:36665.0:2083409.0:7534652....,other_domestic,33963,1749075.0
2,8587837,Sy Ss,Be Ht,Be Ht,wickets,4.0,Sy Ss,bat,Be Ct Gd,Brisbane,...,171.0,6.0,119.0,33956,7869987.0:7620283.0:2076192.0:4002340.0:306369...,33921,7620269.0:2286437.0:87191.0:5786766.0:3114803....,other_domestic,33921,3890984.0
3,8638041,Nn Ds,Oo,Oo,wickets,2.0,Oo,field,By Ol,Mount Maunganui,...,156.0,8.0,126.0,17982,7907451.0:4381761.0:31464.0:4949790.0:258649.0...,18360,2319638.0:256080.0:7918280.0:3913447.0:2690498...,other_domestic,18360,NaN
4,8587921,Ht Hs,Me Ss,Ht Hs,runs,21.0,Me Ss,field,Be Ol,Hobart,...,143.0,9.0,123.0,33928,4223883.0:2161599.0:1655436.0:5788418.0:319948...,33949,363047.0:2275097.0:3901078.0:2275195.0:4230127...,other_domestic,33928,3125849.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,9128601,Pb Ks,Ci Sr Ks,Pb Ks,runs,11.0,Ci Sr Ks,field,We Sm Mi,Mumbai,...,176.0,6.0,126.0,30407,2789079.0:197658.0:2398346.0:2827327.0:2082044...,30414,7422673.0:249087.0:3519011.0:5001170.0:232000....,In Pr Le,30407,197658.0
1685,9433241,Mx,St,St,wickets,9.0,St,field,Te Cr As Cy Gd Tn,Taunton,...,142.0,2.0,81.0,8700,4421689.0:7752989.0:1941743.0:4489722.0:767287...,9701,6139370.0:7694581.0:3294444.0:3239102.0:632036...,other_domestic,9701,NaN
1686,9097227,Bd,Wn Pe,Wn Pe,wickets,6.0,Bd,bat,St Gs Pk Pt Eh,Port Elizabeth,...,143.0,4.0,105.0,22497,4239773.0:1941743.0:3007969.0:4172972.0:155625...,23869,323049.0:4876122.0:4164978.0:1837205.0:3373138...,other_domestic,23869,323049.0
1687,9516695,Rn Rs,Ss Hd,Ss Hd,wickets,4.0,Rn Rs,bat,Si Mh Sm Jr,Jaipur,...,217.0,6.0,125.0,30428,8058959.0:2162782.0:2981614.0:4690188.0:212569...,36014,5958840.0:7491294.0:3127354.0:3057312.0:420349...,In Pr Le,36014,5764576.0


In [13]:
# Feature: Win by Runs/Wickets
# df_match['win_by_runs'] = df_match['by'].apply(lambda x: 'runs' if 'runs' in x else 'wickets')

# Feature: Batting First/Second Performance
def batting_first_win_rate(team_id, data):
    matches_batting_first = data[((data['team1_id'] == team_id) & (data['by'] == 'runs')) |
                                 ((data['team2_id'] == team_id) & (data['by'] == 'wickets'))]
    wins = matches_batting_first[((matches_batting_first['winner'] == get_team1_name(df_match, team_id)) & (matches_batting_first['team1_id'] == team_id)) |
                                 ((matches_batting_first['winner'] == get_team2_name(df_match, team_id)) & (matches_batting_first['team2_id'] == team_id))].shape[0]
    return wins / matches_batting_first.shape[0] if matches_batting_first.shape[0] > 0 else 0

# def batting_second_win_rate(team_id, data):
#     matches_batting_second = data[((data['team1_id'] == team_id) & (data['by'] == 'wickets')) |
#                                   ((data['team2_id'] == team_id) & (data['by'] == 'runs'))]
#     wins = matches_batting_second[((matches_batting_second['winner'] == get_team1_name(df_match, team_id)) & (matches_batting_second['team1_id'] == team_id)) |
#                                   ((matches_batting_second['winner'] == get_team2_name(df_match, team_id)) & (matches_batting_second['team2_id'] == team_id))].shape[0]
#     return wins / matches_batting_second.shape[0] if matches_batting_second.shape[0] > 0 else 0

data['team1_batting_first_win_rate'] = data['team1_id'].apply(lambda x: batting_first_win_rate(x, df_match))
data['team2_batting_first_win_rate'] = data['team2_id'].apply(lambda x: batting_first_win_rate(x, df_match))
# data['team1_batting_second_win_rate'] = data['team1_id'].apply(lambda x: batting_second_win_rate(x, df_match))
# data['team2_batting_second_win_rate'] = data['team2_id'].apply(lambda x: batting_second_win_rate(x, df_match))

# # Feature: Toss Decision Impact
# def toss_decision_impact(team_id, data):
#     toss_wins = data[((data['toss winner'] ==  get_team1_name(df_match, team_id)) & (data['team1_id'] == team_id)) |
#                      ((data['toss winner'] ==  get_team2_name(df_match, team_id)) & (data['team2_id'] == team_id))]
#     toss_wins_and_match_wins = toss_wins[((toss_wins['winner'] ==  get_team1_name(df_match, team_id)) & (toss_wins['team1_id'] == team_id)) |
#                                          ((toss_wins['winner'] ==  get_team2_name(df_match, team_id)) & (toss_wins['team2_id'] == team_id))].shape[0]
#     return toss_wins_and_match_wins / toss_wins.shape[0] if toss_wins.shape[0] > 0 else 0

# data['team1_toss_impact'] = data['team1_id'].apply(lambda x: toss_decision_impact(x, df_match))
# data['team2_toss_impact'] = data['team2_id'].apply(lambda x: toss_decision_impact(x, df_match))


In [14]:
# # Feature: Average Winning Margin
# def average_winning_margin_by_runs(team_id, data):
#     wins_by_runs = data[((data['winner'] == get_team1_name(df_match, team_id)) & (data['team1_id'] == team_id) & (data['by'] == 'runs')) |
#                         ((data['winner'] == get_team2_name(df_match, team_id)) & (data['team2_id'] == team_id) & (data['by'] == 'runs'))].shape[0]
#     return wins_by_runs
#     return wins_by_runs['by'].apply(lambda x: int(x.split(' ')[0])).mean() if not wins_by_runs.empty else 0

# def average_winning_margin_by_wickets(team_id, data):
#     wins_by_wickets = data[((data['winner'] == get_team1_name(df_match, team_id)) & (data['team1_id'] == team_id) & (data['by'] == 'wickets')) |
#                            ((data['winner'] == get_team2_name(df_match, team_id)) & (data['team2_id'] == team_id) & (data['by'] == 'wickets'))].shape[0]
#     return wins_by_wickets

# data['team1_avg_winning_margin_runs'] = data['team1_id'].apply(lambda x: average_winning_margin_by_runs(x, df_match))
# data['team2_avg_winning_margin_runs'] = data['team2_id'].apply(lambda x: average_winning_margin_by_runs(x, df_match))
# data['team1_avg_winning_margin_wickets'] = data['team1_id'].apply(lambda x: average_winning_margin_by_wickets(x, df_match))
# data['team2_avg_winning_margin_wickets'] = data['team2_id'].apply(lambda x: average_winning_margin_by_wickets(x, df_match))

In [15]:
def win_rate(team_id, data, condition):
    matches = data[((data['team1_id'] == team_id) | (data['team2_id'] == team_id)) & (data['lighting'] == condition)]
    wins = matches[((matches['winner'] == get_team1_name(df_match, team_id)) & (matches['team1_id'] == team_id)) |
                   ((matches['winner'] == get_team2_name(df_match, team_id)) & (matches['team2_id'] == team_id))].shape[0]
    return wins / matches.shape[0] if matches.shape[0] > 0 else 0

# Calculate win rates for night, day/night, and day matches for both teams
for condition in ['night match', 'day/night match', 'day match']:
    data[f'team1_{condition}_win_rate'] = data['team1_id'].apply(lambda x: win_rate(x, df_match, condition))
    data[f'team2_{condition}_win_rate'] = data['team2_id'].apply(lambda x: win_rate(x, df_match, condition))

In [16]:
data

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,...,team1_recent_performance,team2_recent_performance,team1_batting_first_win_rate,team2_batting_first_win_rate,team1_night match_win_rate,team2_night match_win_rate,team1_day/night match_win_rate,team2_day/night match_win_rate,team1_day match_win_rate,team2_day match_win_rate
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,Hl Ph,12634,Hl Ph,...,0.4,0.8,1.000,1.0,0.500000,0.000000,0.333333,1.000000,0.642857,0.727273
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409....,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:161392...,Ed,20,Wt Is,...,0.2,0.6,1.000,1.0,0.633333,0.470588,0.444444,0.500000,0.625000,0.428571
2,9433269,We,10576,3298427.0:2288789.0:7773338.0:3519011.0:368195...,Ne,8987,4003390.0:1749075.0:1626526.0:4172447.0:551672...,We,10576,Ne,...,0.6,0.8,1.000,1.0,0.333333,0.571429,0.400000,0.625000,0.500000,0.714286
3,9587073,Ga An Ws,36084,8127230.0:4690328.0:4069666.0:7960847.0:469018...,Bs Rs,36070,3462080.0:2436405.0:1798705.0:7550857.0:574247...,Ga An Ws,36084,Ga An Ws,...,0.8,0.4,1.000,1.0,0.666667,0.357143,0.000000,0.000000,0.500000,0.636364
4,9516457,Pb Ks,30407,8127181.0:197658.0:4239038.0:2398346.0:5053082...,Gt Ts,48341,1958683.0:7491224.0:8059029.0:4377610.0:225245...,Gt Ts,48341,Gt Ts,...,0.4,0.4,1.000,1.0,0.458333,0.529412,0.333333,0.857143,0.500000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9128601,Pb Ks,30407,2789079.0:197658.0:2398346.0:2827327.0:2082044...,Ci Sr Ks,30414,7422673.0:249087.0:3519011.0:5001170.0:232000....,Pb Ks,30407,Ci Sr Ks,...,0.4,0.8,1.000,1.0,0.458333,0.620690,0.333333,0.400000,0.500000,0.500000
944,9433241,Mx,8700,4421689.0:7752989.0:1941743.0:4489722.0:767287...,St,9701,6139370.0:7694581.0:3294444.0:3239102.0:632036...,St,9701,St,...,0.2,0.8,0.875,1.0,0.000000,0.611111,0.181818,0.687500,0.714286,1.000000
945,9097227,Bd,22497,4239773.0:1941743.0:3007969.0:4172972.0:155625...,Wn Pe,23869,323049.0:4876122.0:4164978.0:1837205.0:3373138...,Wn Pe,23869,Bd,...,0.4,0.2,1.000,1.0,0.250000,0.333333,0.000000,0.000000,0.625000,0.625000
946,9516695,Rn Rs,30428,8058959.0:2162782.0:2981614.0:4690188.0:212569...,Ss Hd,36014,5958840.0:7491294.0:3127354.0:3057312.0:420349...,Ss Hd,36014,Rn Rs,...,0.6,0.2,1.000,1.0,0.481481,0.269231,0.600000,0.500000,0.000000,0.000000


In [17]:
def calculate_bowler_ratings(df_bowler):
    df = df_bowler.copy()

    def replace_inf_nan(df, columns, replace_with=100):
        for col in columns:
            df[col] = df[col].replace([np.inf, -np.inf], np.nan).fillna(replace_with)
        return df

    # 1. Calculate basic statistics
    df['bowling_avg'] = np.where(df.groupby('bowler_id')['wicket_count'].transform('sum') != 0,
                                 df.groupby('bowler_id')['runs'].transform('sum') / df.groupby('bowler_id')['wicket_count'].transform('sum'),
                                 100)

    df['strike_rate'] = np.where(df.groupby('bowler_id')['wicket_count'].transform('sum') != 0,
                                 df.groupby('bowler_id')['balls_bowled'].transform('sum') / df.groupby('bowler_id')['wicket_count'].transform('sum'),
                                 100)

    # 2. Calculate dot ball percentage
    df['dot_percentage'] = np.where(df['balls_bowled'] != 0, df['dots'] / df['balls_bowled'], 0)

    # 3. Calculate boundary concession rate
    df['boundary_runs'] = (df['Fours'] * 4) + (df['Sixes'] * 6)
    df['boundary_rate'] = np.where(df['balls_bowled'] != 0, df['boundary_runs'] / df['balls_bowled'], 50)

    # 4. Calculate wicket-taking ability
    df['wicket_rate'] = np.where(df['balls_bowled'] != 0, df['wicket_count'] / (df['balls_bowled'] / 6), 0)

    # 5. Calculate extras control
    df['extras_rate'] = np.where(df['balls_bowled'] != 0, (df['wides'] + df['noballs']) / df['balls_bowled'], 50)

    # 6. Calculate recent form (last 5 matches)
    df = df.sort_values(['bowler_id', 'match_dt'])
    df['recent_form_wickets'] = df.groupby('bowler_id')['wicket_count'].transform(lambda x: x.rolling(5, min_periods=1).mean())
    df['recent_form_economy'] = df.groupby('bowler_id')['economy'].transform(lambda x: x.rolling(5, min_periods=1).mean())

    # 7. Calculate consistency (using coefficient of variation of economy)
    df['economy_std'] = df.groupby('bowler_id')['economy'].transform('std')
    df['consistency'] = np.where(df['economy'] != 0, 1 - (df['economy_std'] / df['economy']), 0)

    # 8. Create the bowler rating
    df['bowler_rating'] = (
        20 * (1 / df['bowling_avg']) +  # Lower average is better
        15 * (1 / df['strike_rate']) +  # Lower strike rate is better
        15 * df['dot_percentage'] +
        15 * (1 - df['boundary_rate']) +  # Lower boundary rate is better
        10 * df['wicket_rate'] +
        10 * (1 - df['extras_rate']) +  # Lower extras rate is better
        5 * df['recent_form_wickets'] +
        5 * (10 - df['recent_form_economy']) +  # Lower economy is better
        5 * df['consistency']
    )

    # Replace infinities and NaNs
    columns_to_check = ['bowling_avg', 'strike_rate', 'dot_percentage', 'boundary_rate',
                        'wicket_rate', 'extras_rate', 'recent_form_wickets',
                        'recent_form_economy', 'consistency', 'bowler_rating']

    df = replace_inf_nan(df, columns_to_check)

    # 9. Normalize the rating (0-100 scale)
    min_rating = df['bowler_rating'].min()
    max_rating = df['bowler_rating'].max()
    if min_rating == max_rating:
        df['normalized_bowler_rating'] = 50  # or any other default value
    else:
        df['normalized_bowler_rating'] = 100 * (df['bowler_rating'] - min_rating) / (max_rating - min_rating)

    # 10. Get the final rating for each bowler as a dictionary
    bowler_ratings = df.groupby('bowler_id')['normalized_bowler_rating'].last().dropna().to_dict()

    return bowler_ratings

# Usage:
# Assuming df_bowler is your input DataFrame
bowler_ratings = calculate_bowler_ratings(df_bowler)


In [18]:
import pandas as pd
import numpy as np

def calculate_batsman_ratings(df_batsman):
    df = df_batsman.copy()

    # Helper function to replace inf and nan
    def replace_inf_nan(series, replace_with=0):
        return series.replace([np.inf, -np.inf], np.nan).fillna(replace_with)

    # 1. Calculate basic statistics
    df['batting_avg'] = df.groupby('batsman_id')['runs'].transform('mean')
    df['strike_rate'] = np.where(df['balls_faced'] != 0, (df['runs'] / df['balls_faced']) * 100, 0)

    # 2. Calculate boundary percentage
    df['boundary_runs'] = (df['Fours'] * 4) + (df['Sixes'] * 6)
    df['boundary_percentage'] = np.where(df['runs'] != 0, df['boundary_runs'] / df['runs'], 0)

    # 3. Calculate consistency (using coefficient of variation)
    df['runs_std'] = df.groupby('batsman_id')['runs'].transform('std')
    df['consistency'] = np.where(df['batting_avg'] != 0, 1 - (df['runs_std'] / df['batting_avg']), 0)

    # 4. Calculate recent form (last 5 matches)
    df = df.sort_values(['batsman_id', 'match_dt'])
    df['recent_form'] = df.groupby('batsman_id')['runs'].transform(lambda x: x.rolling(5, min_periods=1).mean())

    # 5. Calculate power hitting ability
    df['power_hitting'] = np.where(df['balls_faced'] != 0, (df['Fours'] + 2*df['Sixes']) / df['balls_faced'], 0)

    # 6. Create the power rating
    df['power_rating'] = (
        0.3 * replace_inf_nan(df['batting_avg']) +
        0.2 * replace_inf_nan(df['strike_rate']) +
        0.15 * replace_inf_nan(df['boundary_percentage'] * 100) +
        0.15 * replace_inf_nan(df['consistency'] * 100) +
        0.1 * replace_inf_nan(df['recent_form']) +
        0.1 * replace_inf_nan(df['power_hitting'] * 100)
    )

    # 7. Normalize the power rating (0-100 scale)
    min_rating = df['power_rating'].min()
    max_rating = df['power_rating'].max()
    if min_rating == max_rating:
        df['normalized_power_rating'] = 50  # or any other default value
    else:
        df['normalized_power_rating'] = 100 * (df['power_rating'] - min_rating) / (max_rating - min_rating)

    # 8. Get the final power rating for each batsman as a dictionary
    batsman_ratings = df.groupby('batsman_id')['normalized_power_rating'].last().to_dict()

    return batsman_ratings

# Usage:
# Assuming df_batsman is your input DataFrame
batsman_ratings = calculate_batsman_ratings(df_batsman)

In [19]:
def calculate_team_ratings(df, batsman_ratings_dict, bowler_ratings_dict):
    def get_top_5_average(roster_ids, ratings_dict):
        # Convert roster_ids to a list if it's a string
        if isinstance(roster_ids, str):
            roster_ids = roster_ids.split(':')

        # Get ratings for players in the roster, handling float IDs
        ratings = [ratings_dict.get(int(float(player_id)), 0) for player_id in roster_ids if player_id and player_id != 'nan']

        # Sort ratings and get top 5 average
        top_5_avg = np.mean(sorted(ratings, reverse=True)[:5]) if ratings else 0
        return top_5_avg

    # Calculate ratings for each team
    df['team1_batsman_rating'] = df['team1_roster_ids'].apply(lambda x: get_top_5_average(x, batsman_ratings))
    df['team1_bowler_rating'] = df['team1_roster_ids'].apply(lambda x: get_top_5_average(x, bowler_ratings))
    df['team2_batsman_rating'] = df['team2_roster_ids'].apply(lambda x: get_top_5_average(x, batsman_ratings))
    df['team2_bowler_rating'] = df['team2_roster_ids'].apply(lambda x: get_top_5_average(x, bowler_ratings))

    return df

# Usage:
# Assuming df is your input DataFrame, and you have batsman_ratings_dict and bowler_ratings_dict
data = calculate_team_ratings(data, batsman_ratings, bowler_ratings)

In [20]:
print(data.shape)

(948, 41)


## Add avg_team1_data and ground_data

In [21]:
# Merge team1_data with data on team1_id
merged_data_team1 = pd.merge(data, team1_data, left_on='team1_id', right_on='team1_id', suffixes=('', '_team1'),how='outer')

# Merge team1_data with the result on team2_id
merged_data_team2 = pd.merge(merged_data_team1, team1_data, left_on='team2_id', right_on='team1_id', suffixes=('', '_team2'), how='outer')

final_merged_data = pd.merge(merged_data_team2, ground_data, on='ground_id', suffixes=('', '_ground'), how='outer')
final_merged_data.dropna(subset=['match id'], inplace = True)
final_merged_data
# Display the final merged DataFrame
print(final_merged_data.columns)
data=final_merged_data
data

Index(['match id', 'team1', 'team1_id', 'team1_roster_ids', 'team2',
       'team2_id', 'team2_roster_ids', 'winner', 'winner_id', 'toss winner',
       'toss decision', 'venue', 'city', 'match_dt', 'lighting', 'series_name',
       'season', 'ground_id', 'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'team1_win_rate', 'team2_win_rate',
       'team1_venue_performance', 'team2_venue_performance',
       'team1_recent_performance', 'team2_recent_performance',
       'team1_batting_first_win_rate', 'team2_batting_first_win_rate',
       'team1_night match_win_rate', 'team2_night match_win_rate',
       'team1_day/night match_win_rate', 'team2_day/night match_win_rate',
       'team1_day match_win_rate', 'team2_day match_win_rate',
       'team1_batsman_rating', 'team1_bowler_rating', 'team2_batsman_rating',
       'team2_bowler_rating', 'avg_sixes', 'avg_fours', 'avg_wicket',
       'avg_runs'

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,...,avg_runs,avg_sixes_team2,avg_fours_team2,avg_wicket_team2,avg_runs_team2,team1_id_team2,avg_sixes_ground,avg_fours_ground,avg_wicket_ground,avg_runs_ground
0,9085236.0,Ex,7258.0,3200973.0:6065548.0:5406064.0:4489974.0:636458...,Mx,8700.0,4421689.0:4489722.0:7752989.0:150093.0:7671166...,Ex,7258.0,Mx,...,183.714286,3.200000,6.600000,8.800000,151.00,8700.0,2.545455,5.181818,6.636364,152.090909
1,9433563.0,Ex,7258.0,4489974.0:6065548.0:5406064.0:8044490.0:636458...,Mx,8700.0,4421689.0:7752989.0:4005406.0:4489722.0:150093...,Ex,7258.0,Mx,...,183.714286,3.200000,6.600000,8.800000,151.00,8700.0,2.545455,5.181818,6.636364,152.090909
2,9085159.0,Kt,8056.0,87191.0:4655384.0:7787772.0:2083409.0:1707621....,Mx,8700.0,4421689.0:4489722.0:7752989.0:150093.0:7671166...,Mx,8700.0,Kt,...,159.800000,3.200000,6.600000,8.800000,151.00,8700.0,2.545455,5.181818,6.636364,152.090909
3,9433031.0,Sy,9876.0,6282856.0:1613919.0:4640824.0:3851658.0:357372...,Mx,8700.0,4421689.0:7752989.0:1941743.0:4489722.0:400540...,Sy,9876.0,Sy,...,174.538462,3.200000,6.600000,8.800000,151.00,8700.0,2.545455,5.181818,6.636364,152.090909
4,9094441.0,Mr Os,46766.0,4685568.0:1798705.0:325814.0:4171859.0:3501889...,Tt Rs,46731.0,1749075.0:1655436.0:3294444.0:130437.0:1626526...,Tt Rs,46731.0,Mr Os,...,148.666667,3.166667,5.000000,6.000000,140.50,46731.0,2.545455,5.181818,6.636364,152.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9249162.0,Ch Rc,895.0,8392194.0:9198405.0:8095317.0:8095324.0:919841...,Aa,300.0,8392145.0:8392131.0:8096451.0:7326220.0:771249...,Aa,300.0,Aa,...,157.000000,NaN,NaN,NaN,NaN,NaN,3.000000,2.000000,6.000000,157.000000
944,9793818.0,An,293.0,8127363.0:2939124.0:2228393.0:4433575.0:246434...,Si La,69.0,8363221.0:7195082.0:7520344.0:4857838.0:671085...,An,293.0,Si La,...,117.250000,2.100000,4.100000,6.100000,135.40,69.0,3.750000,4.500000,7.500000,101.500000
945,9793811.0,Pn,62.0,7882426.0:9320597.0:7646204.0:8180570.0:829833...,Hg Kg,146.0,2454416.0:4433960.0:3775141.0:8952971.0:643668...,Pn,62.0,Hg Kg,...,130.375000,2.666667,4.000000,5.666667,149.00,146.0,3.750000,4.500000,7.500000,101.500000
946,9793846.0,Pn,62.0,9320597.0:5017144.0:7882426.0:7646204.0:852242...,Bh,188.0,4879426.0:6551978.0:5752256.0:4403426.0:577180...,Bh,188.0,Bh,...,130.375000,1.625000,3.250000,4.875000,95.75,188.0,3.750000,4.500000,7.500000,101.500000


## other preprocessing

In [22]:
# Replace winner_id values based on team1_id and team2_id
data['toss winner'] = data.apply(lambda row: 1 if row['toss winner'] == row['team1'] else 2, axis=1)

In [23]:
# Drop multiple columns in one line, ignoring any columns that do not exist
columns_to_drop = [ 'team1_roster_ids', 'team2_roster_ids', 'team1', 'team2', 'winner','toss decision', 'series_name', 'venue', 'city','team1_id_team2','lighting','season']
data.drop(columns=columns_to_drop, errors='ignore', inplace=True)

In [24]:

# Function to fill NaN with mode and handle ties
def fill_nan_with_mode(df):
    mode_values = df.mode().iloc[0]  # Calculate mode values

    # Fill NaN values with mode
    df_filled = df.apply(lambda x: x.fillna(mode_values[x.name]))

    return df_filled

# Fill NaN values with mode in the DataFrame
data = fill_nan_with_mode(data)
print(data.columns)
data

Index(['match id', 'team1_id', 'team2_id', 'winner_id', 'toss winner',
       'match_dt', 'ground_id', 'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'team1_win_rate', 'team2_win_rate',
       'team1_venue_performance', 'team2_venue_performance',
       'team1_recent_performance', 'team2_recent_performance',
       'team1_batting_first_win_rate', 'team2_batting_first_win_rate',
       'team1_night match_win_rate', 'team2_night match_win_rate',
       'team1_day/night match_win_rate', 'team2_day/night match_win_rate',
       'team1_day match_win_rate', 'team2_day match_win_rate',
       'team1_batsman_rating', 'team1_bowler_rating', 'team2_batsman_rating',
       'team2_bowler_rating', 'avg_sixes', 'avg_fours', 'avg_wicket',
       'avg_runs', 'avg_sixes_team2', 'avg_fours_team2', 'avg_wicket_team2',
       'avg_runs_team2', 'avg_sixes_ground', 'avg_fours_ground',
       'avg_wicket_ground', 

,match id,team1_id,team2_id,winner_id,toss winner,match_dt,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,...,avg_wicket,avg_runs,avg_sixes_team2,avg_fours_team2,avg_wicket_team2,avg_runs_team2,avg_sixes_ground,avg_fours_ground,avg_wicket_ground,avg_runs_ground
0,9085236.0,7258.0,8700.0,7258.0,2,2022-06-23,83.0,1.090909,3.857143,161.600000,...,7.142857,183.714286,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
1,9433563.0,7258.0,8700.0,7258.0,2,2023-06-18,83.0,1.857143,81.000000,176.800000,...,7.142857,183.714286,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
2,9085159.0,8056.0,8700.0,8700.0,1,2022-06-19,83.0,0.800000,0.512195,162.000000,...,6.800000,159.800000,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
3,9433031.0,9876.0,8700.0,9876.0,1,2023-05-25,83.0,0.545455,1.000000,164.333333,...,6.769231,174.538462,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
4,9094441.0,46766.0,46731.0,46731.0,1,2022-09-03,83.0,0.588235,0.672131,154.545455,...,5.833333,148.666667,3.166667,5.000000,6.000000,140.50,2.545455,5.181818,6.636364,152.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9249162.0,895.0,300.0,300.0,2,2022-07-09,24828.0,1.000000,1.000000,165.400000,...,6.000000,157.000000,3.000000,4.000000,6.000000,170.00,3.000000,2.000000,6.000000,157.000000
944,9793818.0,293.0,69.0,293.0,2,2023-10-04,25479.0,0.400000,1.000000,149.000000,...,6.333333,117.250000,2.100000,4.100000,6.100000,135.40,3.750000,4.500000,7.500000,101.500000
945,9793811.0,62.0,146.0,62.0,2,2023-10-03,25479.0,1.666667,1.196078,153.933333,...,6.687500,130.375000,2.666667,4.000000,5.666667,149.00,3.750000,4.500000,7.500000,101.500000
946,9793846.0,62.0,188.0,188.0,2,2023-10-07,25479.0,0.375000,0.506173,151.333333,...,6.687500,130.375000,1.625000,3.250000,4.875000,95.75,3.750000,4.500000,7.500000,101.500000


In [25]:
# Replace winner_id values based on team1_id and team2_id
data['winner_id'] = data.apply(lambda row: 1 if row['winner_id'] == row['team1_id'] else 2, axis=1)

In [26]:
# Check which columns have NaN values
columns_with_nan = data.columns[data.isna().any()]

print("Columns with NaN values:")
print(columns_with_nan)

Columns with NaN values:
Index([], dtype='object')


In [27]:
# Identify categorical columns
cat_columns = data.select_dtypes(include=['object']).columns
cat_columns

# # Encode categorical variables (excluding winner_id, team1_id, and team2_id)
encoder_dict = {}
for col in cat_columns:
    if col not in ['winner_id', 'team1_id', 'team2_id']:
        encoder = LabelEncoder()
        data[col] = encoder.fit_transform(data[col])
        encoder_dict[col] = encoder


# # Split the dataset into features and target
x = data.drop(columns=[ 'winner_id'])
y = data['winner_id']
x

,match id,team1_id,team2_id,toss winner,match_dt,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,...,avg_wicket,avg_runs,avg_sixes_team2,avg_fours_team2,avg_wicket_team2,avg_runs_team2,avg_sixes_ground,avg_fours_ground,avg_wicket_ground,avg_runs_ground
0,9085236.0,7258.0,8700.0,2,124,83.0,1.090909,3.857143,161.600000,66.67,...,7.142857,183.714286,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
1,9433563.0,7258.0,8700.0,2,383,83.0,1.857143,81.000000,176.800000,75.00,...,7.142857,183.714286,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
2,9085159.0,8056.0,8700.0,1,121,83.0,0.800000,0.512195,162.000000,100.00,...,6.800000,159.800000,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
3,9433031.0,9876.0,8700.0,1,363,83.0,0.545455,1.000000,164.333333,100.00,...,6.769231,174.538462,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
4,9094441.0,46766.0,46731.0,1,177,83.0,0.588235,0.672131,154.545455,0.00,...,5.833333,148.666667,3.166667,5.000000,6.000000,140.50,2.545455,5.181818,6.636364,152.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9249162.0,895.0,300.0,2,133,24828.0,1.000000,1.000000,165.400000,0.00,...,6.000000,157.000000,3.000000,4.000000,6.000000,170.00,3.000000,2.000000,6.000000,157.000000
944,9793818.0,293.0,69.0,2,441,25479.0,0.400000,1.000000,149.000000,0.00,...,6.333333,117.250000,2.100000,4.100000,6.100000,135.40,3.750000,4.500000,7.500000,101.500000
945,9793811.0,62.0,146.0,2,440,25479.0,1.666667,1.196078,153.933333,0.00,...,6.687500,130.375000,2.666667,4.000000,5.666667,149.00,3.750000,4.500000,7.500000,101.500000
946,9793846.0,62.0,188.0,2,443,25479.0,0.375000,0.506173,151.333333,100.00,...,6.687500,130.375000,1.625000,3.250000,4.875000,95.75,3.750000,4.500000,7.500000,101.500000


In [28]:
y

0      1
1      1
2      2
3      1
4      2
      ..
943    2
944    1
945    1
946    2
947    2
Name: winner_id, Length: 948, dtype: int64

In [29]:
list(data.columns)

['match id',
 'team1_id',
 'team2_id',
 'winner_id',
 'toss winner',
 'match_dt',
 'ground_id',
 'team_count_50runs_last15',
 'team_winp_last5',
 'team1only_avg_runs_last15',
 'team1_winp_team2_last15',
 'ground_avg_runs_last15',
 'team1_win_rate',
 'team2_win_rate',
 'team1_venue_performance',
 'team2_venue_performance',
 'team1_recent_performance',
 'team2_recent_performance',
 'team1_batting_first_win_rate',
 'team2_batting_first_win_rate',
 'team1_night match_win_rate',
 'team2_night match_win_rate',
 'team1_day/night match_win_rate',
 'team2_day/night match_win_rate',
 'team1_day match_win_rate',
 'team2_day match_win_rate',
 'team1_batsman_rating',
 'team1_bowler_rating',
 'team2_batsman_rating',
 'team2_bowler_rating',
 'avg_sixes',
 'avg_fours',
 'avg_wicket',
 'avg_runs',
 'avg_sixes_team2',
 'avg_fours_team2',
 'avg_wicket_team2',
 'avg_runs_team2',
 'avg_sixes_ground',
 'avg_fours_ground',
 'avg_wicket_ground',
 'avg_runs_ground']

In [30]:
data

,match id,team1_id,team2_id,winner_id,toss winner,match_dt,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,...,avg_wicket,avg_runs,avg_sixes_team2,avg_fours_team2,avg_wicket_team2,avg_runs_team2,avg_sixes_ground,avg_fours_ground,avg_wicket_ground,avg_runs_ground
0,9085236.0,7258.0,8700.0,1,2,124,83.0,1.090909,3.857143,161.600000,...,7.142857,183.714286,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
1,9433563.0,7258.0,8700.0,1,2,383,83.0,1.857143,81.000000,176.800000,...,7.142857,183.714286,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
2,9085159.0,8056.0,8700.0,2,1,121,83.0,0.800000,0.512195,162.000000,...,6.800000,159.800000,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
3,9433031.0,9876.0,8700.0,1,1,363,83.0,0.545455,1.000000,164.333333,...,6.769231,174.538462,3.200000,6.600000,8.800000,151.00,2.545455,5.181818,6.636364,152.090909
4,9094441.0,46766.0,46731.0,2,1,177,83.0,0.588235,0.672131,154.545455,...,5.833333,148.666667,3.166667,5.000000,6.000000,140.50,2.545455,5.181818,6.636364,152.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9249162.0,895.0,300.0,2,2,133,24828.0,1.000000,1.000000,165.400000,...,6.000000,157.000000,3.000000,4.000000,6.000000,170.00,3.000000,2.000000,6.000000,157.000000
944,9793818.0,293.0,69.0,1,2,441,25479.0,0.400000,1.000000,149.000000,...,6.333333,117.250000,2.100000,4.100000,6.100000,135.40,3.750000,4.500000,7.500000,101.500000
945,9793811.0,62.0,146.0,1,2,440,25479.0,1.666667,1.196078,153.933333,...,6.687500,130.375000,2.666667,4.000000,5.666667,149.00,3.750000,4.500000,7.500000,101.500000
946,9793846.0,62.0,188.0,2,2,443,25479.0,0.375000,0.506173,151.333333,...,6.687500,130.375000,1.625000,3.250000,4.875000,95.75,3.750000,4.500000,7.500000,101.500000


In [31]:
# Drop multiple columns in one line, ignoring any columns that do not exist
columns_to_drop = [ 'avg_runs_ground', 'avg_fours_ground', 'avg_runs_team2', 'avg_wicket_ground', 'ground_avg_runs_last15','ground_id', 'team1_day match_win_rate', 'team1_day/night match_win_rate', 'team1_night match_win_rate','team2_day/night match_win_rate','toss winner']
data.drop(columns=columns_to_drop, errors='ignore', inplace=True)

In [32]:
data

,match id,team1_id,team2_id,winner_id,match_dt,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,team1_win_rate,...,team2_batsman_rating,team2_bowler_rating,avg_sixes,avg_fours,avg_wicket,avg_runs,avg_sixes_team2,avg_fours_team2,avg_wicket_team2,avg_sixes_ground
0,9085236.0,7258.0,8700.0,1,124,1.090909,3.857143,161.600000,66.67,0.540541,...,45.485342,64.011805,3.428571,5.071429,7.142857,183.714286,3.200000,6.600000,8.800000,2.545455
1,9433563.0,7258.0,8700.0,1,383,1.857143,81.000000,176.800000,75.00,0.540541,...,52.461033,61.775991,3.428571,5.071429,7.142857,183.714286,3.200000,6.600000,8.800000,2.545455
2,9085159.0,8056.0,8700.0,2,121,0.800000,0.512195,162.000000,100.00,0.500000,...,45.485342,66.132093,3.200000,5.300000,6.800000,159.800000,3.200000,6.600000,8.800000,2.545455
3,9433031.0,9876.0,8700.0,1,363,0.545455,1.000000,164.333333,100.00,0.600000,...,47.228021,62.274907,3.461538,4.923077,6.769231,174.538462,3.200000,6.600000,8.800000,2.545455
4,9094441.0,46766.0,46731.0,2,177,0.588235,0.672131,154.545455,0.00,0.333333,...,46.812960,73.239803,3.333333,4.166667,5.833333,148.666667,3.166667,5.000000,6.000000,2.545455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9249162.0,895.0,300.0,2,133,1.000000,1.000000,165.400000,0.00,0.000000,...,32.541805,74.766513,3.000000,2.000000,6.000000,157.000000,3.000000,4.000000,6.000000,3.000000
944,9793818.0,293.0,69.0,1,441,0.400000,1.000000,149.000000,0.00,0.448276,...,31.874011,75.595934,2.666667,4.166667,6.333333,117.250000,2.100000,4.100000,6.100000,3.750000
945,9793811.0,62.0,146.0,1,440,1.666667,1.196078,153.933333,0.00,0.603774,...,31.968594,79.508939,2.625000,4.687500,6.687500,130.375000,2.666667,4.000000,5.666667,3.750000
946,9793846.0,62.0,188.0,2,443,0.375000,0.506173,151.333333,100.00,0.603774,...,27.088173,75.369232,2.625000,4.687500,6.687500,130.375000,1.625000,3.250000,4.875000,3.750000


In [33]:
x = data.drop(columns=[ 'winner_id'])
y = data['winner_id']

In [34]:
x

,match id,team1_id,team2_id,match_dt,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,team1_win_rate,team2_win_rate,...,team2_batsman_rating,team2_bowler_rating,avg_sixes,avg_fours,avg_wicket,avg_runs,avg_sixes_team2,avg_fours_team2,avg_wicket_team2,avg_sixes_ground
0,9085236.0,7258.0,8700.0,124,1.090909,3.857143,161.600000,66.67,0.540541,0.259259,...,45.485342,64.011805,3.428571,5.071429,7.142857,183.714286,3.200000,6.600000,8.800000,2.545455
1,9433563.0,7258.0,8700.0,383,1.857143,81.000000,176.800000,75.00,0.540541,0.259259,...,52.461033,61.775991,3.428571,5.071429,7.142857,183.714286,3.200000,6.600000,8.800000,2.545455
2,9085159.0,8056.0,8700.0,121,0.800000,0.512195,162.000000,100.00,0.500000,0.259259,...,45.485342,66.132093,3.200000,5.300000,6.800000,159.800000,3.200000,6.600000,8.800000,2.545455
3,9433031.0,9876.0,8700.0,363,0.545455,1.000000,164.333333,100.00,0.600000,0.259259,...,47.228021,62.274907,3.461538,4.923077,6.769231,174.538462,3.200000,6.600000,8.800000,2.545455
4,9094441.0,46766.0,46731.0,177,0.588235,0.672131,154.545455,0.00,0.333333,0.666667,...,46.812960,73.239803,3.333333,4.166667,5.833333,148.666667,3.166667,5.000000,6.000000,2.545455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9249162.0,895.0,300.0,133,1.000000,1.000000,165.400000,0.00,0.000000,0.500000,...,32.541805,74.766513,3.000000,2.000000,6.000000,157.000000,3.000000,4.000000,6.000000,3.000000
944,9793818.0,293.0,69.0,441,0.400000,1.000000,149.000000,0.00,0.448276,0.425000,...,31.874011,75.595934,2.666667,4.166667,6.333333,117.250000,2.100000,4.100000,6.100000,3.750000
945,9793811.0,62.0,146.0,440,1.666667,1.196078,153.933333,0.00,0.603774,0.375000,...,31.968594,79.508939,2.625000,4.687500,6.687500,130.375000,2.666667,4.000000,5.666667,3.750000
946,9793846.0,62.0,188.0,443,0.375000,0.506173,151.333333,100.00,0.603774,0.418605,...,27.088173,75.369232,2.625000,4.687500,6.687500,130.375000,1.625000,3.250000,4.875000,3.750000


In [35]:
y

0      1
1      1
2      2
3      1
4      2
      ..
943    2
944    1
945    1
946    2
947    2
Name: winner_id, Length: 948, dtype: int64

## Train Model


In [36]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(X_train.columns)
# Define models with adjusted hyperparameters
models = {
    'GBM': GradientBoostingClassifier(n_estimators=30, learning_rate=0.1, random_state=42),
    'CatBoost': cb.CatBoostClassifier(iterations=30, learning_rate=0.1, depth=10, random_state=42, verbose=0)
}

# Evaluate each model without RFE
# Evaluate each model without RFE
for name, model in models.items():
    # Train model with all features
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)

    # Print test accuracy every 50 iterations
    for i, y_pred in enumerate(model.staged_predict(X_test)):
        if i % 50 == 0:
            # Calculate test accuracy
            test_accuracy = accuracy_score(y_test, y_pred)
            print(f"Iteration {i+1}: Test Accuracy: {test_accuracy:.4f}")

    # Print predictions along with team1_id, team2_id, and actual winner (y)
    predictions_df = pd.DataFrame({
        'team1_id': X_test['team1_id'],
        'team2_id': X_test['team2_id'],
        'Actual Winner': y_test,
        'Predicted Winner': y_pred.squeeze()
    })

    print(f"{name} Predictions and Actual Winners:")
    print(predictions_df)

    # Evaluate test accuracy
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f"\n{name} Test Accuracy: {test_accuracy:.4f}\n")

Index(['match id', 'team1_id', 'team2_id', 'match_dt',
       'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'team1_win_rate', 'team2_win_rate', 'team1_venue_performance',
       'team2_venue_performance', 'team1_recent_performance',
       'team2_recent_performance', 'team1_batting_first_win_rate',
       'team2_batting_first_win_rate', 'team2_night match_win_rate',
       'team2_day match_win_rate', 'team1_batsman_rating',
       'team1_bowler_rating', 'team2_batsman_rating', 'team2_bowler_rating',
       'avg_sixes', 'avg_fours', 'avg_wicket', 'avg_runs', 'avg_sixes_team2',
       'avg_fours_team2', 'avg_wicket_team2', 'avg_sixes_ground'],
      dtype='object')
Iteration 1: Test Accuracy: 0.8316
GBM Predictions and Actual Winners:
     team1_id  team2_id  Actual Winner  Predicted Winner
589   47508.0   49657.0              2                 1
933     146.0     202.0              1                 2
139   33956.0  

In [37]:
model

In [38]:
train_length = 948

In [39]:
# catboost
trainhpstrees = model.get_all_params()['iterations']
trainhpsdepth = model.get_all_params()['depth']
trainhpslr = model.get_all_params()['learning_rate']

train_hps_trees = pd.DataFrame({"train_hps_trees":  [trainhpstrees] * train_length })
train_hps_depth = pd.DataFrame({"train_hps_depth":  [trainhpsdepth] * train_length})
train_hps_lr = pd.DataFrame({"train_hps_lr":  [trainhpslr] * train_length})
train_hps_trees

,train_hps_trees
0,30
1,30
2,30
3,30
4,30
...,...
943,30
944,30
945,30
946,30


In [40]:
# gbm
# trainhpstrees = model.n_estimators
# trainhpsdepth = model.max_depth
# trainhpslr = model.learning_rate
# train_hps_trees = pd.DataFrame({"train_hps_trees":  [trainhpstrees] * train_length })
# train_hps_depth = pd.DataFrame({"train_hps_depth":  [trainhpsdepth] * train_length})
# train_hps_lr = pd.DataFrame({"train_hps_lr":  [trainhpslr] * train_length})
# train_hps_trees

In [41]:
# catboost
feature_importance = model.get_feature_importance(type="FeatureImportance")

# Normalize to get percentages
total_importance = sum(feature_importance)
feature_importance_percent = (feature_importance / total_importance) * 100

# Sort feature importance in descending order and get the top 10 features
top_10_feature_indices = feature_importance_percent.argsort()[-10:][::-1]
top_10_feature_names = [model.feature_names_[idx] for idx in top_10_feature_indices]
top_10_feature_importance = feature_importance_percent[top_10_feature_indices]



print("Top 10 Features and their Importance (%):")
for name, importance in zip(top_10_feature_names, top_10_feature_importance):
  print(f"{name}: {importance:.2f}%")

indep_feat_id1 = pd.DataFrame({"indep_feat_id1" :  [top_10_feature_importance[0]] * train_length })
indep_feat_id2 = pd.DataFrame({"indep_feat_id2" :  [top_10_feature_importance[1]] * train_length })
indep_feat_id3 = pd.DataFrame({"indep_feat_id3" :  [top_10_feature_importance[2]] * train_length })
indep_feat_id4 = pd.DataFrame({"indep_feat_id4" :  [top_10_feature_importance[3]] * train_length})
indep_feat_id5 = pd.DataFrame({"indep_feat_id5" :  [top_10_feature_importance[4]] * train_length })
indep_feat_id6 = pd.DataFrame({"indep_feat_id6" :  [top_10_feature_importance[5]] * train_length})
indep_feat_id7 = pd.DataFrame({"indep_feat_id7" :  [top_10_feature_importance[6]] * train_length })
indep_feat_id8 = pd.DataFrame({"indep_feat_id8" :  [top_10_feature_importance[7]] * train_length })
indep_feat_id9 = pd.DataFrame({"indep_feat_id9" :  [top_10_feature_importance[8]] * train_length})
indep_feat_id10 = pd.DataFrame({"indep_feat_id10" :  [top_10_feature_importance[9]] * train_length })

dataset_type = pd.DataFrame({"dataset_type": ["train"] * train_length})
is_ensemble = pd.DataFrame({"is_ensemble": ["no"] * train_length})
train_algorithm = pd.DataFrame({"train_algorithm": ["catboost"] * train_length})

Top 10 Features and their Importance (%):
team1_venue_performance: 21.46%
team2_venue_performance: 19.39%
team1_bowler_rating: 3.60%
team1only_avg_runs_last15: 3.57%
team1_batsman_rating: 3.43%
match_dt: 2.91%
team1_winp_team2_last15: 2.87%
team_count_50runs_last15: 2.82%
avg_sixes: 2.68%
avg_fours: 2.46%


In [42]:
# gbm
# # feature names
# feature_names = X_train.columns.tolist()

# # Get feature importances
# importances = model.feature_importances_
# importances
# # Sort features by importance
# feature_importances = sorted(zip(importances, feature_names), reverse=True)

# # Get top 10 features
# top_10_features = feature_importances[:10]

# # Create a DataFrame for better visualization
# df_top_10 = pd.DataFrame(top_10_features, columns=['Importance', 'Feature'])
# df_top_10 = df_top_10.sort_values('Importance', ascending=False).reset_index(drop=True)
# df_top_10
# top_10_feature_importance = df_top_10['Importance']

# indep_feat_id1 = pd.DataFrame({"indep_feat_id1" :  [top_10_feature_importance[0]] * train_length })
# indep_feat_id2 = pd.DataFrame({"indep_feat_id2" :  [top_10_feature_importance[1]] * train_length })
# indep_feat_id3 = pd.DataFrame({"indep_feat_id3" :  [top_10_feature_importance[2]] * train_length })
# indep_feat_id4 = pd.DataFrame({"indep_feat_id4" :  [top_10_feature_importance[3]] * train_length})
# indep_feat_id5 = pd.DataFrame({"indep_feat_id5" :  [top_10_feature_importance[4]] * train_length })
# indep_feat_id6 = pd.DataFrame({"indep_feat_id6" :  [top_10_feature_importance[5]] * train_length})
# indep_feat_id7 = pd.DataFrame({"indep_feat_id7" :  [top_10_feature_importance[6]] * train_length })
# indep_feat_id8 = pd.DataFrame({"indep_feat_id8" :  [top_10_feature_importance[7]] * train_length })
# indep_feat_id9 = pd.DataFrame({"indep_feat_id9" :  [top_10_feature_importance[8]] * train_length})
# indep_feat_id10 = pd.DataFrame({"indep_feat_id10" :  [top_10_feature_importance[9]] * train_length })

# dataset_type = pd.DataFrame({"dataset_type": ["train"] * train_length})
# is_ensemble = pd.DataFrame({"is_ensemble": ["no"] * train_length})
# train_algorithm = pd.DataFrame({"train_algorithm": ["gbm"] * train_length})

In [43]:
scores = model.predict_proba(x)
winning_prediction_scores = np.max(scores, axis=1)
winning_prediction_scores
win_pred_score = pd.DataFrame(winning_prediction_scores)
win_pred_score['win_pred_score']= winning_prediction_scores
win_pred_score = win_pred_score['win_pred_score']
win_pred_score = pd.DataFrame(win_pred_score)
win_pred_score

,win_pred_score
0,0.967517
1,0.947087
2,0.670912
3,0.948551
4,0.878014
...,...
943,0.941936
944,0.942393
945,0.889178
946,0.919383


#train_data_prediction

In [44]:
y = model.predict(x)
y =pd.DataFrame(y)
y['team'] = y
y = y['team']
y = pd.DataFrame(y)


# Loop through each row in df and replace the values
for index, row in y.iterrows():
   if row['team'] == 1:
     y.at[index, 'team'] = x.at[index, 'team1_id']
   elif row['team'] == 2:
    y.at[index, 'team'] = x.at[index, 'team2_id']


win_pred_team_id = pd.DataFrame(y)
win_pred_team_id['win_pred_team_id'] = win_pred_team_id
win_pred_team_id = win_pred_team_id['win_pred_team_id']
win_pred_team_id = pd.DataFrame(win_pred_team_id)
win_pred_team_id

,win_pred_team_id
0,7258
1,7258
2,8700
3,9876
4,46731
...,...
943,300
944,293
945,62
946,188


In [45]:
primary = pd.concat([x['match id'],dataset_type,win_pred_team_id,win_pred_score,train_algorithm,is_ensemble,train_hps_trees,train_hps_depth,train_hps_lr,indep_feat_id1,indep_feat_id2, indep_feat_id3, indep_feat_id4, indep_feat_id5, indep_feat_id6, indep_feat_id7, indep_feat_id8, indep_feat_id9, indep_feat_id10], axis=1)
primary

,match id,dataset_type,win_pred_team_id,win_pred_score,train_algorithm,is_ensemble,train_hps_trees,train_hps_depth,train_hps_lr,indep_feat_id1,indep_feat_id2,indep_feat_id3,indep_feat_id4,indep_feat_id5,indep_feat_id6,indep_feat_id7,indep_feat_id8,indep_feat_id9,indep_feat_id10
0,9085236.0,train,7258,0.967517,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
1,9433563.0,train,7258,0.947087,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
2,9085159.0,train,8700,0.670912,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
3,9433031.0,train,9876,0.948551,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
4,9094441.0,train,46731,0.878014,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9249162.0,train,300,0.941936,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
944,9793818.0,train,293,0.942393,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
945,9793811.0,train,62,0.889178,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
946,9793846.0,train,188,0.919383,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144


#Creating Secondary Submission

In [46]:
#  catboost
feature_descriptions = [
    "team1 avg performance on that ground id",
    "team2 avg performance on that ground id",
    "team1 avg bowler rating",
    "team1's avg inning runs in last 15 games",
    "team1 avg batsman rating",
    "match date",
    "Team1's win percentage against Team2 in last 15 games",
    "Ratio of number of 50s by players in team1 to number of 50s by players in team2 in last 15 games",
    "average sixes scored",
    "average fours scored",
    "team2 avg bowler rating",
    "team2 avg batsman rating",
    "unique id of team1",
    "team1 avg winning rate",
    "unique id of team2",
    "average sixes scored by team2",
    "Recentmost performance of team1",
    "average runs scored",
    "Ratio of team1's win % to team2's win % in last 5 games",
    "team2 win rate in night matches",
    "average wickets taken by team 2",
    "team2 win rate in day matches",
    "Recentmost performance of team2",
    "average wickets taken",
    "unique id of a game",
    "average fours scored by team2",
    "team2 avg winning rate",
    "average sixes scored on the ground",
    "team2 avg winning rate if batting first",
    "team1 avg winning rate if batting first",
     
#     "average runs scored in the ground in last 15 games",
#      "average runs scored on the ground",
#     "unique id of the ground the match is held at",
     
#     "average runs scored by team 2",
#     "unique id of a game",
#     "unique id of team1",
#     "average wickets taken on the ground",

#     "toss winner team",
#     "lighting condition of match - day/night, day or night match",
#     "average fours scored on the ground",
#     "seasons of the series. For e.g., 2021/2022, 2022, etc."
]

feature_descriptions = pd.DataFrame(feature_descriptions, columns =['feat_description'])

feature_importance = model.get_feature_importance(type="FeatureImportance")

# Normalize to get percentages
total_importance = sum(feature_importance)
feature_importance_percent = (feature_importance / total_importance) * 100

# Sort feature importance in descending order and get the top 10 features
feature_indices = feature_importance_percent.argsort()[:][::-1]
# feature_names = [model.feature_names_[idx] for idx in feature_indices]
feature_importance_percetage = feature_importance_percent[feature_indices]



# Create a DataFrame
ranks = np.arange(1, 31)
rank = pd.DataFrame(ranks, columns=['feat_rank_train'])

# Map feature names to importance values
feature_importance_rank = [(idx, model.feature_names_[idx], feature_importance_percent[idx]) for idx in feature_indices]



# Create DataFrame
df = pd.DataFrame(feature_importance_rank, columns=['feat_id', 'feat_name', 'model_feat_imp_train'])
df['feat_rank_train'] = rank
df
# ,feature_descriptions,df['model_feat_imp_train'],df['feat_rank_train']
secondary = pd.DataFrame(df['feat_id'],columns=['feat_id'])
secondary['feat_name']=df['feat_name']
secondary['feat_description']=feature_descriptions
secondary['model_feat_imp_train']=df['model_feat_imp_train']
secondary['feat_rank_train']=df['feat_rank_train']
secondary

,feat_id,feat_name,feat_description,model_feat_imp_train,feat_rank_train
0,10,team1_venue_performance,team1 avg performance on that ground id,21.461995,1
1,11,team2_venue_performance,team2 avg performance on that ground id,19.394031,2
2,19,team1_bowler_rating,team1 avg bowler rating,3.595793,3
3,6,team1only_avg_runs_last15,team1's avg inning runs in last 15 games,3.570957,4
4,18,team1_batsman_rating,team1 avg batsman rating,3.434521,5
5,3,match_dt,match date,2.914523,6
6,7,team1_winp_team2_last15,Team1's win percentage against Team2 in last 1...,2.874325,7
7,4,team_count_50runs_last15,Ratio of number of 50s by players in team1 to ...,2.816964,8
8,22,avg_sixes,average sixes scored,2.675738,9
9,23,avg_fours,average fours scored,2.461144,10


In [47]:
# gbm
# # feature names
# feature_names = X_train.columns.tolist()

# # Get feature importances
# importances = model.feature_importances_
# importances
# # Sort features by importance
# feature_importances = sorted(zip(importances, feature_names), reverse=True)

# # Create a DataFrame for better visualization
# features = pd.DataFrame(feature_importances, columns=['Importance', 'Feature'])
# features = features.sort_values('Importance', ascending=False).reset_index(drop=True)
# features

In [48]:
#download secondary file
secondary.to_csv('secondary', index=False)

# Inference

In [165]:
test_length = 207

# Processing test data

In [166]:
test_data = pd.read_csv(csv_test_filename)

In [167]:
test_data

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,toss winner,toss decision,venue,...,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15
0,9272619,Me Ss,33949,4003390.0:7960994.0:3901078.0:2669316.0:373710...,Ht Hs,33928,5843200.0:4223883.0:4655384.0:6249256.0:216159...,Ht Hs,field,Be Ol Ht,...,2023-01-09,night match,Bg Bh Le,2022/23,6348,0.666667,1.000000,167.933333,60.0,154.115385
1,9086958,Na,209,5836452.0:8246468.0:7500324.0:3065502.0:363350...,Si La,69,7200598.0:4403531.0:3260564.0:2420760.0:239834...,Si La,field,GA Sm Sh Gg Va,...,2022-10-16,day/night match,Si La tr of Aa,2022/23,3961,0.142857,0.207921,141.888889,0.0,142.833333
2,9433654,Gn,7573,5164844.0:8110185.0:2979857.0:1722048.0:270743...,St,9701,6139370.0:7694581.0:3294444.0:3239102.0:481700...,St,field,Sa Gs Cf,...,2023-06-21,day/night match,Vy Bt,2023,4521,0.666667,0.344262,178.800000,0.0,155.733333
3,9097248,Ks,22784,3496933.0:4167673.0:4160316.0:1863889.0:310883...,Ds,22763,3399745.0:4898074.0:3782225.0:2252452.0:223190...,Ks,bat,St Gs Pk Pt Eh,...,2022-02-08,day match,CA T0 Ce,2021/22,1224,0.500000,0.259259,138.000000,0.0,142.500000
4,9097234,Ws,23841,4166882.0:3009215.0:1963443.0:3252927.0:417185...,Ts,23750,2654014.0:2667223.0:3406717.0:3057312.0:415964...,Ts,field,St Gs Pk Pt Eh,...,2022-02-07,day match,CA T0 Ce,2021/22,1224,0.166667,1.000000,155.800000,100.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,9984162,Pb Ks,30407,7881719.0:8127181.0:2231928.0:2642751.0:505308...,Ss Hd,36014,3710090.0:7491294.0:3127354.0:8228485.0:305731...,Pb Ks,bat,Rv Gi Il Sm Ul Hd,...,2024-05-19,day/night match,In Pr Le,2024,13880,0.307692,1.952381,181.400000,50.0,181.625000
203,9984176,Ss Hd,36014,3710090.0:7491294.0:3127354.0:8228485.0:811799...,Ka Kt Rs,30400,6818622.0:1613919.0:5959834.0:4497646.0:506174...,Ss Hd,bat,Na Mi Sm Ad,...,2024-05-21,night match,In Pr Le,2024,5893,0.857143,0.344262,189.133333,40.0,168.033333
204,9984183,Rl Cs Bu,30393,1776627.0:313809.0:7537004.0:5765934.0:2275195...,Rn Rs,30428,8058959.0:3294444.0:2981614.0:7556051.0:822842...,Rn Rs,field,Na Mi Sm Ad,...,2024-05-22,night match,In Pr Le,2024,5893,1.368421,0.672131,185.666667,75.0,168.033333
205,9984190,Ss Hd,36014,3710090.0:7491294.0:3127354.0:4203499.0:305731...,Rn Rs,30428,8058959.0:3294444.0:2981614.0:7556051.0:822842...,Rn Rs,field,MA Cm Sm Ck Ci,...,2024-05-24,night match,In Pr Le,2024,2050,0.736842,0.344262,189.133333,50.0,157.933333


In [168]:
test_data['team1_win_rate'] = test_data['team1_id'].apply(lambda x: calculate_win_rate(x, df_match))
test_data['team2_win_rate'] = test_data['team2_id'].apply(lambda x: calculate_win_rate(x, df_match))

In [169]:
test_data['team1_venue_performance'] = test_data.apply(lambda x: venue_performance(x['team1_id'], x['ground_id'], df_match), axis=1)
test_data['team2_venue_performance'] = test_data.apply(lambda x: venue_performance(x['team2_id'], x['ground_id'], df_match), axis=1)
# Apply the function to calculate recent performance for team1 and team2
test_data['team1_recent_performance'] = test_data['team1_id'].apply(lambda x: recent_performance(x, df_match))
test_data['team2_recent_performance'] = test_data['team2_id'].apply(lambda x: recent_performance(x, df_match))

In [170]:
test_data['team1_batting_first_win_rate'] = test_data['team1_id'].apply(lambda x: batting_first_win_rate(x, df_match))
test_data['team2_batting_first_win_rate'] = test_data['team2_id'].apply(lambda x: batting_first_win_rate(x, df_match))

In [171]:
for condition in ['night match', 'day/night match', 'day match']:
    test_data[f'team1_{condition}_win_rate'] = test_data['team1_id'].apply(lambda x: win_rate(x, df_match, condition))
    test_data[f'team2_{condition}_win_rate'] = test_data['team2_id'].apply(lambda x: win_rate(x, df_match, condition))

In [172]:
test_data = calculate_team_ratings(test_data, batsman_ratings, bowler_ratings)

In [173]:
test_data

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,toss winner,toss decision,venue,...,team1_night match_win_rate,team2_night match_win_rate,team1_day/night match_win_rate,team2_day/night match_win_rate,team1_day match_win_rate,team2_day match_win_rate,team1_batsman_rating,team1_bowler_rating,team2_batsman_rating,team2_bowler_rating
0,9272619,Me Ss,33949,4003390.0:7960994.0:3901078.0:2669316.0:373710...,Ht Hs,33928,5843200.0:4223883.0:4655384.0:6249256.0:216159...,Ht Hs,field,Be Ol Ht,...,0.423077,0.450000,0.333333,0.285714,0.500000,0.000000,31.629420,68.546966,40.590140,65.590418
1,9086958,Na,209,5836452.0:8246468.0:7500324.0:3065502.0:363350...,Si La,69,7200598.0:4403531.0:3260564.0:2420760.0:239834...,Si La,field,GA Sm Sh Gg Va,...,0.200000,0.419355,0.000000,1.000000,0.625000,0.285714,30.774754,70.356112,30.592226,66.411670
2,9433654,Gn,7573,5164844.0:8110185.0:2979857.0:1722048.0:270743...,St,9701,6139370.0:7694581.0:3294444.0:3239102.0:481700...,St,field,Sa Gs Cf,...,0.250000,0.611111,0.333333,0.687500,0.428571,1.000000,42.629122,66.668433,41.115693,64.105317
3,9097248,Ks,22784,3496933.0:4167673.0:4160316.0:1863889.0:310883...,Ds,22763,3399745.0:4898074.0:3782225.0:2252452.0:223190...,Ks,bat,St Gs Pk Pt Eh,...,1.000000,1.000000,0.000000,0.000000,0.285714,0.538462,29.563005,72.099059,46.480620,78.491562
4,9097234,Ws,23841,4166882.0:3009215.0:1963443.0:3252927.0:417185...,Ts,23750,2654014.0:2667223.0:3406717.0:3057312.0:415964...,Ts,field,St Gs Pk Pt Eh,...,1.000000,0.000000,0.000000,0.000000,0.300000,0.750000,38.563970,73.546878,39.565000,74.156980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,9984162,Pb Ks,30407,7881719.0:8127181.0:2231928.0:2642751.0:505308...,Ss Hd,36014,3710090.0:7491294.0:3127354.0:8228485.0:305731...,Pb Ks,bat,Rv Gi Il Sm Ul Hd,...,0.458333,0.269231,0.333333,0.500000,0.500000,0.000000,33.185261,71.982125,44.155995,68.240996
203,9984176,Ss Hd,36014,3710090.0:7491294.0:3127354.0:8228485.0:811799...,Ka Kt Rs,30400,6818622.0:1613919.0:5959834.0:4497646.0:506174...,Ss Hd,bat,Na Mi Sm Ad,...,0.269231,0.531250,0.500000,0.142857,0.000000,0.500000,44.155995,68.240996,43.478393,77.121119
204,9984183,Rl Cs Bu,30393,1776627.0:313809.0:7537004.0:5765934.0:2275195...,Rn Rs,30428,8058959.0:3294444.0:2981614.0:7556051.0:822842...,Rn Rs,field,Na Mi Sm Ad,...,0.551724,0.481481,0.714286,0.600000,1.000000,0.000000,49.431780,71.363742,32.290250,67.680738
205,9984190,Ss Hd,36014,3710090.0:7491294.0:3127354.0:4203499.0:305731...,Rn Rs,30428,8058959.0:3294444.0:2981614.0:7556051.0:822842...,Rn Rs,field,MA Cm Sm Ck Ci,...,0.269231,0.481481,0.500000,0.600000,0.000000,0.000000,42.179322,69.935909,32.290250,67.680738


In [174]:
# Merge team1_data with data on team1_id
merged_data_team1 = pd.merge(test_data, team1_data, left_on='team1_id', right_on='team1_id', suffixes=('', '_team1'), how='outer')

# Merge team1_data with the result on team2_id
merged_data_team2 = pd.merge(merged_data_team1, team1_data, left_on='team2_id', right_on='team1_id', suffixes=('', '_team2'), how='outer')

final_merged_data = pd.merge(merged_data_team2, ground_data, on='ground_id', suffixes=('', '_ground'), how='outer')
final_merged_data.dropna(subset=['match id'], inplace = True)

# # Display the final merged DataFrame
print(final_merged_data.columns)
test_data=final_merged_data
test_data

Index(['match id', 'team1', 'team1_id', 'team1_roster_ids', 'team2',
       'team2_id', 'team2_roster_ids', 'toss winner', 'toss decision', 'venue',
       'city', 'match_dt', 'lighting', 'series_name', 'season', 'ground_id',
       'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'team1_win_rate', 'team2_win_rate',
       'team1_venue_performance', 'team2_venue_performance',
       'team1_recent_performance', 'team2_recent_performance',
       'team1_batting_first_win_rate', 'team2_batting_first_win_rate',
       'team1_night match_win_rate', 'team2_night match_win_rate',
       'team1_day/night match_win_rate', 'team2_day/night match_win_rate',
       'team1_day match_win_rate', 'team2_day match_win_rate',
       'team1_batsman_rating', 'team1_bowler_rating', 'team2_batsman_rating',
       'team2_bowler_rating', 'avg_sixes', 'avg_fours', 'avg_wicket',
       'avg_runs', 'avg_sixes_team2', 'a

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,toss winner,toss decision,venue,...,avg_runs,avg_sixes_team2,avg_fours_team2,avg_wicket_team2,avg_runs_team2,team1_id_team2,avg_sixes_ground,avg_fours_ground,avg_wicket_ground,avg_runs_ground
0,9084991.0,Sy,9876.0,2089079.0:6282856.0:4640824.0:1798705.0:161391...,Mx,8700.0,4421689.0:4489722.0:172199.0:7752989.0:150093....,Mx,field,Ls Ln,...,174.538462,3.200000,6.600000,8.800000,151.000000,8700.0,2.545455,5.181818,6.636364,152.090909
1,9433409.0,Sx,9967.0,7753017.0:3622856.0:4215098.0:74087.0:6460614....,Mx,8700.0,4421689.0:7752989.0:1941743.0:4489722.0:767116...,Sx,bat,Ls Ln,...,164.785714,3.200000,6.600000,8.800000,151.000000,8700.0,2.545455,5.181818,6.636364,152.090909
2,9489409.0,Mr Os,46766.0,4685568.0:2162782.0:4489722.0:1798705.0:350188...,Tt Rs,46731.0,1749075.0:2125696.0:3294444.0:1626526.0:578832...,Tt Rs,field,Tt Be Nm,...,148.666667,3.166667,5.000000,6.000000,140.500000,46731.0,3.437500,5.812500,7.250000,159.500000
3,9433668.0,Sy,9876.0,6282856.0:1798705.0:4640824.0:3573723.0:385165...,Mx,8700.0,4421689.0:7752989.0:4489722.0:4005406.0:767116...,Mx,field,Kn Ol Ln,...,174.538462,3.200000,6.600000,8.800000,151.000000,8700.0,3.125000,4.937500,6.187500,157.187500
4,9084641.0,Gn,7573.0,2531913.0:1722048.0:5515922.0:5164844.0:270743...,Sy,9876.0,2089079.0:6282856.0:6210462.0:1798705.0:464082...,Sy,field,Kn Ol Ln,...,167.800000,3.461538,4.923077,6.769231,174.538462,9876.0,3.125000,4.937500,6.187500,157.187500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,9983910.0,Mi Is,30435.0,5043310.0:238727.0:3125562.0:8191868.0:4377610...,Pb Ks,30407.0,4640824.0:8127181.0:2231928.0:2827327.0:238599...,Pb Ks,field,Ma Ya Sh Il Ct Sm Mr,...,137.900000,3.692308,5.076923,6.538462,144.846154,30407.0,NaN,NaN,NaN,NaN
253,9983840.0,Ss Hd,36014.0,3710090.0:7491294.0:4203499.0:8228485.0:312735...,Pb Ks,30407.0,197658.0:2082044.0:8127181.0:4640824.0:2097017...,Pb Ks,field,Ma Ya Sh Il Ct Sm Mr,...,183.200000,3.692308,5.076923,6.538462,144.846154,30407.0,NaN,NaN,NaN,NaN
254,9983868.0,Pb Ks,30407.0,7881719.0:2082044.0:8127181.0:4640824.0:505308...,Rn Rs,30428.0,8058959.0:7881733.0:2981614.0:7556051.0:822842...,Rn Rs,field,Ma Ya Sh Il Ct Sm Mr,...,144.846154,3.166667,4.166667,5.250000,170.833333,30428.0,NaN,NaN,NaN,NaN
255,9983938.0,Pb Ks,30407.0,4640824.0:8127181.0:2231928.0:5053082.0:282732...,Gt Ts,48341.0,1958683.0:7491224.0:8059029.0:2252452.0:573601...,Pb Ks,bat,Ma Ya Sh Il Ct Sm Mr,...,144.846154,2.666667,4.583333,5.416667,170.000000,48341.0,NaN,NaN,NaN,NaN


In [175]:
# Replace winner_id values based on team1_id and team2_id
test_data['toss winner'] = test_data.apply(lambda row: 1 if row['toss winner'] == row['team1'] else 2, axis=1)


In [176]:
# Drop multiple columns in one line, ignoring any columns that do not exist
columns_to_drop = [ 'team1_roster_ids', 'team2_roster_ids', 'team1', 'team2', 'winner','toss decision', 'series_name', 'venue', 'city','team1_id_team2']
test_data.drop(columns=columns_to_drop, errors='ignore', inplace=True)

In [177]:
# Fill NaN values with mode in the DataFrame
test_data = fill_nan_with_mode(test_data)
test_data

,match id,team1_id,team2_id,toss winner,match_dt,lighting,season,ground_id,team_count_50runs_last15,team_winp_last5,...,avg_wicket,avg_runs,avg_sixes_team2,avg_fours_team2,avg_wicket_team2,avg_runs_team2,avg_sixes_ground,avg_fours_ground,avg_wicket_ground,avg_runs_ground
0,9084991.0,9876.0,8700.0,2,2022-06-09,day/night match,2022,83.0,1.545455,1.975610,...,6.769231,174.538462,3.200000,6.600000,8.800000,151.000000,2.545455,5.181818,6.636364,152.090909
1,9433409.0,9967.0,8700.0,1,2023-06-08,day/night match,2023,83.0,0.454545,21.000000,...,7.142857,164.785714,3.200000,6.600000,8.800000,151.000000,2.545455,5.181818,6.636364,152.090909
2,9489409.0,46766.0,46731.0,2,2023-08-17,night match,2023,251.0,1.000000,0.506173,...,5.833333,148.666667,3.166667,5.000000,6.000000,140.500000,3.437500,5.812500,7.250000,159.500000
3,9433668.0,9876.0,8700.0,2,2023-06-22,day/night match,2023,328.0,1.857143,81.000000,...,6.769231,174.538462,3.200000,6.600000,8.800000,151.000000,3.125000,4.937500,6.187500,157.187500
4,9084641.0,7573.0,9876.0,2,2022-05-27,day/night match,2022,328.0,0.600000,0.512195,...,6.600000,167.800000,3.461538,4.923077,6.769231,174.538462,3.125000,4.937500,6.187500,157.187500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,9983910.0,30435.0,30407.0,2,2024-04-18,night match,2024,25108.0,4.250000,1.975610,...,5.500000,137.900000,3.692308,5.076923,6.538462,144.846154,4.000000,4.500000,5.500000,163.700000
253,9983840.0,36014.0,30407.0,2,2024-04-09,night match,2024,25108.0,1.076923,0.512195,...,5.600000,183.200000,3.692308,5.076923,6.538462,144.846154,4.000000,4.500000,5.500000,163.700000
254,9983868.0,30407.0,30428.0,2,2024-04-13,night match,2024,25108.0,0.444444,0.672131,...,6.538462,144.846154,3.166667,4.166667,5.250000,170.833333,4.000000,4.500000,5.500000,163.700000
255,9983938.0,30407.0,48341.0,1,2024-04-21,night match,2024,25108.0,0.230769,1.000000,...,6.538462,144.846154,2.666667,4.583333,5.416667,170.000000,4.000000,4.500000,5.500000,163.700000


In [178]:
# Check which columns have NaN values
columns_with_nan_test = test_data.columns[test_data.isna().any()]

print("Columns with NaN values:")
print(columns_with_nan_test)
cat_columns_test = test_data.select_dtypes(include=['object']).columns
cat_columns_test

# # Encode categorical variables (excluding winner_id, team1_id, and team2_id)
encoder_dict = {}
for col in cat_columns_test:
    if col not in ['team1_id', 'team2_id']:
        encoder = LabelEncoder()
        test_data[col] = encoder.fit_transform(test_data[col])
        encoder_dict[col] = encoder


# # Split the dataset into features and target
x_test = test_data
x_test

Columns with NaN values:
Index([], dtype='object')


,match id,team1_id,team2_id,toss winner,match_dt,lighting,season,ground_id,team_count_50runs_last15,team_winp_last5,...,avg_wicket,avg_runs,avg_sixes_team2,avg_fours_team2,avg_wicket_team2,avg_runs_team2,avg_sixes_ground,avg_fours_ground,avg_wicket_ground,avg_runs_ground
0,9084991.0,9876.0,8700.0,2,31,1,1,83.0,1.545455,1.975610,...,6.769231,174.538462,3.200000,6.600000,8.800000,151.000000,2.545455,5.181818,6.636364,152.090909
1,9433409.0,9967.0,8700.0,1,94,1,3,83.0,0.454545,21.000000,...,7.142857,164.785714,3.200000,6.600000,8.800000,151.000000,2.545455,5.181818,6.636364,152.090909
2,9489409.0,46766.0,46731.0,2,103,2,3,251.0,1.000000,0.506173,...,5.833333,148.666667,3.166667,5.000000,6.000000,140.500000,3.437500,5.812500,7.250000,159.500000
3,9433668.0,9876.0,8700.0,2,98,1,3,328.0,1.857143,81.000000,...,6.769231,174.538462,3.200000,6.600000,8.800000,151.000000,3.125000,4.937500,6.187500,157.187500
4,9084641.0,7573.0,9876.0,2,25,1,1,328.0,0.600000,0.512195,...,6.600000,167.800000,3.461538,4.923077,6.769231,174.538462,3.125000,4.937500,6.187500,157.187500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,9983910.0,30435.0,30407.0,2,152,2,5,25108.0,4.250000,1.975610,...,5.500000,137.900000,3.692308,5.076923,6.538462,144.846154,4.000000,4.500000,5.500000,163.700000
253,9983840.0,36014.0,30407.0,2,143,2,5,25108.0,1.076923,0.512195,...,5.600000,183.200000,3.692308,5.076923,6.538462,144.846154,4.000000,4.500000,5.500000,163.700000
254,9983868.0,30407.0,30428.0,2,147,2,5,25108.0,0.444444,0.672131,...,6.538462,144.846154,3.166667,4.166667,5.250000,170.833333,4.000000,4.500000,5.500000,163.700000
255,9983938.0,30407.0,48341.0,1,155,2,5,25108.0,0.230769,1.000000,...,6.538462,144.846154,2.666667,4.583333,5.416667,170.000000,4.000000,4.500000,5.500000,163.700000


In [179]:
# Drop multiple columns in one line, ignoring any columns that do not exist
columns_to_drop = [ 'avg_runs_ground', 'avg_fours_ground', 'avg_runs_team2', 'avg_wicket_ground', 'ground_avg_runs_last15','ground_id', 'team1_day match_win_rate', 'team1_day/night match_win_rate', 'team1_night match_win_rate','team2_day/night match_win_rate','toss winner','lighting','season']
test_data.drop(columns=columns_to_drop, errors='ignore', inplace=True)

In [180]:
test_data

,match id,team1_id,team2_id,match_dt,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,team1_win_rate,team2_win_rate,...,team2_batsman_rating,team2_bowler_rating,avg_sixes,avg_fours,avg_wicket,avg_runs,avg_sixes_team2,avg_fours_team2,avg_wicket_team2,avg_sixes_ground
0,9084991.0,9876.0,8700.0,31,1.545455,1.975610,160.214286,100.0,0.600000,0.259259,...,42.216199,64.663642,3.461538,4.923077,6.769231,174.538462,3.200000,6.600000,8.800000,2.545455
1,9433409.0,9967.0,8700.0,94,0.454545,21.000000,176.733333,50.0,0.441176,0.259259,...,51.489548,63.209400,3.642857,5.500000,7.142857,164.785714,3.200000,6.600000,8.800000,2.545455
2,9489409.0,46766.0,46731.0,103,1.000000,0.506173,150.666667,0.0,0.333333,0.666667,...,43.588644,71.686839,3.333333,4.166667,5.833333,148.666667,3.166667,5.000000,6.000000,3.437500
3,9433668.0,9876.0,8700.0,98,1.857143,81.000000,190.466667,100.0,0.600000,0.259259,...,52.461033,66.957703,3.461538,4.923077,6.769231,174.538462,3.200000,6.600000,8.800000,3.125000
4,9084641.0,7573.0,9876.0,25,0.600000,0.512195,165.461538,50.0,0.333333,0.600000,...,43.897884,69.641265,4.400000,5.200000,6.600000,167.800000,3.461538,4.923077,6.769231,3.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,9983910.0,30435.0,30407.0,152,4.250000,1.975610,179.266667,50.0,0.483871,0.437500,...,31.372385,75.221697,3.200000,4.400000,5.500000,137.900000,3.692308,5.076923,6.538462,4.000000
253,9983840.0,36014.0,30407.0,143,1.076923,0.512195,189.133333,50.0,0.312500,0.437500,...,43.521589,68.290675,3.800000,4.800000,5.600000,183.200000,3.692308,5.076923,6.538462,4.000000
254,9983868.0,30407.0,30428.0,147,0.444444,0.672131,181.400000,40.0,0.437500,0.484848,...,29.855396,68.624484,3.692308,5.076923,6.538462,144.846154,3.166667,4.166667,5.250000,4.000000
255,9983938.0,30407.0,48341.0,155,0.230769,1.000000,181.400000,0.0,0.437500,0.625000,...,38.189506,68.195274,3.692308,5.076923,6.538462,144.846154,2.666667,4.583333,5.416667,4.000000


In [181]:
x_test = test_data

In [182]:
#download secondary file
x_test.to_csv('x_test', index=False)

In [183]:
predictions = model.predict(x_test)
predictions= pd.DataFrame(predictions,columns=['team'])
predictions

,team
0,1
1,2
2,2
3,1
4,2
...,...
202,2
203,2
204,2
205,2


In [ ]:
# # Loop through each row in df and replace the values
# for index, row in predictions.iterrows():
#    if row['team'] == 1:
#     predictions.at[index, 'team'] = x_test.at[index, 'team1_id']
#    elif row['team'] == 2:
#     predictions.at[index, 'team'] = x_test.at[index, 'team2_id']


# win_pred_team_id = pd.DataFrame(predictions)
# win_pred_team_id['win_pred_team_id'] = win_pred_team_id
# win_pred_team_id = win_pred_team_id['win_pred_team_id']
# win_pred_team_id = pd.DataFrame(win_pred_team_id)
# win_pred_team_id

# Ensure both DataFrames have the same index
predictions = predictions.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

# Loop through each row in predictions and replace the values
for index, row in predictions.iterrows():
    if row['team'] == 1 and index in x_test.index:
        predictions.at[index, 'team'] = x_test.at[index, 'team1_id']
    elif row['team'] == 2 and index in x_test.index:
        predictions.at[index, 'team'] = x_test.at[index, 'team2_id']

win_pred_team_id = predictions[['team']].copy()
win_pred_team_id.columns = ['win_pred_team_id']

win_pred_team_id

In [ ]:
scores = model.predict_proba(x_test)
winning_prediction_scores = np.max(scores, axis=1)
winning_prediction_scores
win_pred_score = pd.DataFrame(winning_prediction_scores)
win_pred_score['win_pred_score']= winning_prediction_scores
win_pred_score = win_pred_score['win_pred_score']
win_pred_score = pd.DataFrame(win_pred_score)
win_pred_score

In [ ]:
train_hps_trees = pd.DataFrame({"train_hps_trees":  [trainhpstrees] * test_length})
train_hps_depth = pd.DataFrame({"train_hps_depth":  [trainhpsdepth] * test_length})
train_hps_lr = pd.DataFrame({"train_hps_lr":  [trainhpslr] * test_length})
dataset_type = pd.DataFrame({"dataset_type": ["r2"] * test_length})
is_ensemble = pd.DataFrame({"is_ensemble": ["no"] * test_length})
train_algorithm = pd.DataFrame({"train_algorithm": ["catboost"] * test_length})

In [ ]:
indep_feat_id1 = pd.DataFrame({"indep_feat_id1" :  [top_10_feature_importance[0]] * test_length })
indep_feat_id2 = pd.DataFrame({"indep_feat_id2" :  [top_10_feature_importance[1]] * test_length })
indep_feat_id3 = pd.DataFrame({"indep_feat_id3" :  [top_10_feature_importance[2]] * test_length })
indep_feat_id4 = pd.DataFrame({"indep_feat_id4" :  [top_10_feature_importance[3]] * test_length})
indep_feat_id5 = pd.DataFrame({"indep_feat_id5" :  [top_10_feature_importance[4]] * test_length })
indep_feat_id6 = pd.DataFrame({"indep_feat_id6" :  [top_10_feature_importance[5]] * test_length})
indep_feat_id7 = pd.DataFrame({"indep_feat_id7" :  [top_10_feature_importance[6]] * test_length })
indep_feat_id8 = pd.DataFrame({"indep_feat_id8" :  [top_10_feature_importance[7]] * test_length })
indep_feat_id9 = pd.DataFrame({"indep_feat_id9" :  [top_10_feature_importance[8]] * test_length})
indep_feat_id10 = pd.DataFrame({"indep_feat_id10" :  [top_10_feature_importance[9]] * test_length })

In [202]:
primary_test = pd.concat([x_test['match id'],dataset_type,win_pred_team_id,win_pred_score,train_algorithm,is_ensemble,train_hps_trees,train_hps_depth,train_hps_lr, indep_feat_id1, indep_feat_id2, indep_feat_id3, indep_feat_id4, indep_feat_id5, indep_feat_id6, indep_feat_id7, indep_feat_id8, indep_feat_id9, indep_feat_id10], axis=1)
primary_test

,match id,dataset_type,win_pred_team_id,win_pred_score,train_algorithm,is_ensemble,train_hps_trees,train_hps_depth,train_hps_lr,indep_feat_id1,indep_feat_id2,indep_feat_id3,indep_feat_id4,indep_feat_id5,indep_feat_id6,indep_feat_id7,indep_feat_id8,indep_feat_id9,indep_feat_id10
0,9084991.0,r2,9876,0.953423,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
1,9433409.0,r2,8700,0.733970,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
2,9489409.0,r2,46731,0.935776,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
3,9433668.0,r2,9876,0.928740,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
4,9084641.0,r2,9876,0.930393,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,9983910.0,r2,30407,0.575443,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
203,9983840.0,r2,30407,0.659192,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
204,9983868.0,r2,30428,0.510758,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
205,9983938.0,r2,48341,0.658505,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144


In [203]:
data = pd.read_csv(csv_filename)

#Creating Primary Submission

In [204]:
test_data = pd.read_csv(csv_test_filename)

In [197]:
# primary_match_id = data['match id']
# primary_match_id = pd.DataFrame(primary_match_id, columns =['match id'])
primary_test_match_id = test_data['match id']
primary_test_match_id = pd.DataFrame(primary_test_match_id, columns =['match id'])

In [205]:
# primary_sorted= primary_match_id.merge(primary,on='match id')
primary_test_sorted= primary_test_match_id.merge(primary_test,on='match id')

In [206]:
primary_submission = pd.concat([primary_test_sorted], axis=0)
primary_submission

,match id,dataset_type,win_pred_team_id,win_pred_score,train_algorithm,is_ensemble,train_hps_trees,train_hps_depth,train_hps_lr,indep_feat_id1,indep_feat_id2,indep_feat_id3,indep_feat_id4,indep_feat_id5,indep_feat_id6,indep_feat_id7,indep_feat_id8,indep_feat_id9,indep_feat_id10
0,9272619,r2,33928,0.633625,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
1,9086958,r2,69,0.926137,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
2,9433654,r2,9701,0.840555,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
3,9097248,r2,22763,0.530496,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
4,9097234,r2,23750,0.940744,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,9984162,r2,36014,0.612765,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
203,9984176,r2,30400,0.891673,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
204,9984183,r2,30428,0.570358,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
205,9984190,r2,36014,0.706073,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144


In [207]:
primary_submission['match id'] = primary_submission['match id'].astype(int)
primary_submission

,match id,dataset_type,win_pred_team_id,win_pred_score,train_algorithm,is_ensemble,train_hps_trees,train_hps_depth,train_hps_lr,indep_feat_id1,indep_feat_id2,indep_feat_id3,indep_feat_id4,indep_feat_id5,indep_feat_id6,indep_feat_id7,indep_feat_id8,indep_feat_id9,indep_feat_id10
0,9272619,r2,33928,0.633625,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
1,9086958,r2,69,0.926137,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
2,9433654,r2,9701,0.840555,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
3,9097248,r2,22763,0.530496,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
4,9097234,r2,23750,0.940744,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,9984162,r2,36014,0.612765,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
203,9984176,r2,30400,0.891673,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
204,9984183,r2,30428,0.570358,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144
205,9984190,r2,36014,0.706073,catboost,no,30,10,0.1,21.461995,19.394031,3.595793,3.570957,3.434521,2.914523,2.874325,2.816964,2.675738,2.461144


In [208]:
primary_submission.to_csv('primary_submission', index=False)